# Analysis Oseen time
In this notebook we calculate the Oseen time based on the slip velocity of the particles

In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

import numpy as np
import xarray as xr
from datetime import datetime, timedelta
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import colormaps
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import Divider, Size
plt.style.use('../python_style_Meike.mplstyle')
import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")


Magma = colormaps['magma']
Magmalist = Magma(np.linspace(0.2, 0.9, 5))
from analysis_functions import make_lognormal_PDF
from particle_characteristics_functions import factor_drag_white1991, Re_particle, dynamic_viscosity_Sharqawy, Oseen_time, stokes_relaxation_time

In [ ]:
#settings
av_temp_NWES = 11.983276
av_salinity_NWES = 34.70449
rho_water = 1027 # kg/m3 
dynamic_viscosity_water = dynamic_viscosity_Sharqawy(av_temp_NWES,av_salinity_NWES/1000)# 1.e-3# 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water
diameter = 0.25 # m
B=0.68
tau=3196


starttime = datetime(2023,9,1,0,0,0)
runtime = timedelta(days =2)
endtime = starttime + runtime

loc = 'NWES'
land_handling = 'anti_beaching'
Replist= [0,10,100,450,1000]
chunck_time = 100
nparticles = 52511
coriolis = True
gradient = True
dt = timedelta(minutes=5).seconds

In [ ]:
# read in data
base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'
basefile_Rep_constant = (base_directory + '{particle_type}/{loc}_'
                 'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                 'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_'
                 '_Rep_{Rep:04d}_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefile_Rep_drag = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefiles={
           'inertial_Rep_constant':basefile_Rep_constant,
           'inertial_SM_Rep_constant':basefile_Rep_constant, 
           'inertial_SM_drag_Rep':basefile_Rep_drag,
           'inertial_drag_Rep':basefile_Rep_drag}

particle_types = ['inertial_drag_Rep','inertial_Rep_constant','inertial_SM_Rep_constant']#,'inertial_Rep_constant'] # 
names = {'inertial_Rep_constant':'MR',
         'inertial_SM_Rep_constant':'SM MR', 
         'inertial_SM_drag_Rep':'SM MR flexible Re$_p$',
         'inertial_drag_Rep':'MR flexible Re$_p$'}
Rep_dict = {'inertial_Rep_constant':Replist,
         'inertial_SM_Rep_constant':Replist,
         'inertial_SM_drag_Rep':[None],
         'inertial_drag_Rep':[None]}


# import data
# create data directory
data = {}
for pt in particle_types:
    data[pt]={}


for pt in particle_types:
    for Rep in Rep_dict[pt]:
        file = basefiles[pt].format(loc=loc,
                                   y_s=starttime.year,
                                   m_s=starttime.month,
                                   d_s=starttime.day,
                                   y_e=endtime.year,
                                   m_e=endtime.month,
                                   d_e=endtime.day,
                                   B = int(B * 1000), 
                                   tau = int(tau ),
                                   land_handling = land_handling, 
                                   coriolis = coriolis,
                                   gradient = gradient,
                                   particle_type = pt,
                                   Rep = Rep)
        ds = xr.open_dataset(file,
                             engine='zarr',
                             chunks={'trajectory':nparticles, 'obs':chunck_time},
                             drop_variables=['B','tau','z'],
                             decode_times=False) 
        print(file)

        
        
        data[pt][Rep]= ds 

In [ ]:
legend = []
fig, ax = plt.subplots()
for Rep,color in zip(Replist[0:],Magmalist[0:]):
    Uslip = np.sqrt(data['inertial_Rep_constant'][Rep].uslip**2 + data['inertial_Rep_constant'][Rep].vslip**2)
    toseen_values =Oseen_time(Uslip.isel(obs=slice(1,None)),kinematic_viscosity_water)
    toseen_values =toseen_values.values.flatten()
    toseen_values = toseen_values[~np.isnan(toseen_values)]
    bins, pdfs = make_lognormal_PDF(toseen_values,nbins=100,norm=True,vmin=5e-6,vmax=5e3)
    legend.append('Re$_p^{\\mathrm{In}}=$'+f'{Rep}')
    ax.plot(bins[1:],pdfs[1:]* (bins[1:]-bins[:-1]),color=color)
   
Uslip = np.sqrt(data['inertial_drag_Rep'][None].uslip**2 + data['inertial_drag_Rep'][None].vslip**2)
toseen_values =Oseen_time(Uslip.isel(obs=slice(1,None)),kinematic_viscosity_water)
toseen_values =toseen_values.values.flatten()
toseen_values = toseen_values[~np.isnan(toseen_values)]
bins, pdfs = make_lognormal_PDF(toseen_values,nbins=100,norm=True,vmin=5e-6,vmax=5e3)
legend.append('flexible Re$_p$')
ax.plot(bins[1:],pdfs[1:]* (bins[1:]-bins[:-1]),color='cornflowerblue')
ax.set_xscale('log')
ax.legend(legend)
ax.set_ylabel('PDF')
ax.set_xlabel('Oseen time [s]')
fig.tight_layout()
fig.savefig('../figures/history_force/PDF_oseen_time.pdf')

In [ ]:
# calculate X percentile values
X = 95
i=1
fig, ax = plt.subplots()
five_percent_list_to = []
for Rep,color in zip(Replist,Magmalist): 
    Uslip = np.sqrt(data['inertial_Rep_constant'][Rep].uslip**2 + data['inertial_Rep_constant'][Rep].vslip**2).isel(obs=slice(1,None))
    uslip =Uslip.values.flatten()
    
    to=Oseen_time(np.nanpercentile(uslip,100-X),kinematic_viscosity_water)
    print(f'tOseen({Rep})= {to:.3f}')
    five_percent_list_to.append(to)
    ax.plot(i,to,'s',color=color)
    i+=1
Rep = None
Uslip = np.sqrt(data['inertial_drag_Rep'][None].uslip**2 + data['inertial_drag_Rep'][None].vslip**2).isel(obs=slice(1,None))
uslip =Uslip.values.flatten()

to=Oseen_time(np.nanpercentile(uslip,100-X),kinematic_viscosity_water)
print(f'tOseen({Rep})= {to:.3f}')
five_percent_list_to.append(to)
ax.plot(i,to,'o',color='cornflowerblue')

ax.set_xticks([1,2,3,4,5,6])
ax.set_xticklabels(['0','10','100','450','1000','flexible'])
ax.set_yscale('log')
ax.set_xlabel('Re$_p^{\\mathrm{In}}$')
ax.set_ylabel(f'{X}-percentile Oseen time [s]')
fig.tight_layout()
fig.savefig(f'../figures/history_force/{X:03d}_percentile_oseen_time.pdf')